In [1]:
import sqlite3
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
con = sqlite3.connect('matches.db')

In [3]:
df_matches =  pd.read_sql_query("SELECT * FROM match", con)

In [4]:
df_matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1580 entries, 0 to 1579
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         1580 non-null   int64 
 1   game_id    1580 non-null   object
 2   team_1     1580 non-null   object
 3   team_2     1580 non-null   object
 4   gold_diff  1580 non-null   object
 5   winner     1580 non-null   object
dtypes: int64(1), object(5)
memory usage: 74.2+ KB


In [5]:
df_matches.sample(5)

,id,game_id,team_1,team_2,gold_diff,winner
1480,1481,NA1_4185852950,"[2500, 2500, 2875, 4350, 6142, 8675, 10350, 11...","[2500, 2500, 2868, 4790, 5952, 7725, 9357, 108...","[0, 0, 7, -440, 190, 950, 993, 909, 1291, 3635...",Team 2
1345,1346,NA1_4185826797,"[2500, 2500, 2903, 4479, 5984, 7864, 9253, 114...","[2500, 2500, 2986, 4440, 5974, 7365, 8836, 103...","[0, 0, -83, 39, 10, 499, 417, 1104, 885, 1341,...",Team 2
1261,1262,NA1_4187383071,"[2500, 2500, 2895, 5142, 6857, 8143, 10773, 12...","[2500, 2500, 2945, 4386, 6236, 7740, 9395, 108...","[0, 0, -50, 756, 621, 403, 1378, 1307, 2098, 3...",Team 2
50,51,NA1_4184548460,"[2500, 2500, 2982, 4351, 5838, 7313, 8349, 975...","[2500, 2500, 2994, 4608, 7281, 8517, 11067, 12...","[0, 0, -12, -257, -1443, -1204, -2718, -3198, ...",Team 1
1426,1427,NA1_4187105585,"[2500, 2520, 3005, 5353, 6381, 8188, 9913, 116...","[2500, 2540, 2887, 4152, 5834, 7473, 8448, 979...","[0, -20, 118, 1201, 547, 715, 1465, 1861, 2404...",Team 2


In [6]:
df_matches = df_matches.drop_duplicates(subset=['game_id'])

In [7]:
df_matches.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 844 entries, 0 to 1579
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         844 non-null    int64 
 1   game_id    844 non-null    object
 2   team_1     844 non-null    object
 3   team_2     844 non-null    object
 4   gold_diff  844 non-null    object
 5   winner     844 non-null    object
dtypes: int64(1), object(5)
memory usage: 46.2+ KB


In [8]:
column_names = ["Minute 0", "Minute 1", "Minute 2", "Minute 3", "Minute 4", "Minute 5", "Minute 6", "Minute 7", "Minute 8", "Minute 9", "Minute 10", "Minute 11", "Minute 12", "Minute 13", "Minute 14",  "Minute 15",  "Minute 16",  "Minute 17",  "Minute 18",  "Minute 19",  "Minute 20", "Winner" ]
df_gold_diff = pd.DataFrame()

In [9]:
df_gold_db = df_matches[["gold_diff", "winner"]]

In [10]:
for index, row in df_gold_db.iterrows():
    gold_reg_mod = [float(item) for item in row["gold_diff"].strip("[]'").strip(" '' ").split(", ")]
    
    if len(gold_reg_mod) > 21:
        a_row = gold_reg_mod[:21]
        a_row.append(row["winner"])
        a_row = pd.Series(a_row)
        row_df = pd.DataFrame([a_row])
        df_gold_diff = pd.concat([row_df, df_gold_diff], ignore_index=True)

In [11]:
df_gold_diff

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,0.0,0.0,34.0,-515.0,612.0,223.0,1070.0,477.0,713.0,111.0,...,2189.0,5157.0,5393.0,4762.0,5830.0,8715.0,9169.0,11533.0,10730.0,Team 2
1,0.0,0.0,554.0,461.0,-70.0,-794.0,-670.0,-1290.0,-1022.0,-934.0,...,1671.0,121.0,-1608.0,1355.0,2958.0,4630.0,3835.0,4840.0,3968.0,Team 2
2,0.0,0.0,1513.0,1401.0,1056.0,1883.0,2036.0,2513.0,3388.0,4162.0,...,6717.0,5454.0,6365.0,7320.0,7419.0,7512.0,9531.0,8937.0,8863.0,Team 2
3,0.0,0.0,-102.0,-326.0,-615.0,-1593.0,-745.0,-1507.0,-1156.0,-1766.0,...,-2846.0,-2867.0,-4020.0,-5429.0,-8722.0,-9003.0,-9332.0,-9346.0,-9660.0,Team 1
4,0.0,0.0,827.0,568.0,157.0,-378.0,-534.0,-1456.0,-1372.0,-1764.0,...,329.0,261.0,1210.0,-344.0,789.0,663.0,-324.0,-1282.0,-346.0,Team 2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
714,0.0,-10.0,-63.0,-57.0,555.0,109.0,696.0,-123.0,-643.0,-859.0,...,-131.0,664.0,1754.0,741.0,1230.0,332.0,2268.0,3488.0,3757.0,Team 2
715,0.0,0.0,32.0,-36.0,-517.0,-748.0,-867.0,-1408.0,-973.0,-1187.0,...,-1595.0,-476.0,942.0,-854.0,-982.0,-383.0,853.0,208.0,314.0,Team 1
716,0.0,0.0,12.0,-682.0,-309.0,1111.0,-83.0,-614.0,561.0,585.0,...,1402.0,1527.0,1194.0,279.0,644.0,-84.0,731.0,-1992.0,-467.0,Team 2
717,0.0,519.0,518.0,105.0,-133.0,-204.0,-663.0,900.0,768.0,174.0,...,-1674.0,-2542.0,-1768.0,-1137.0,-1898.0,-2845.0,-5912.0,-6809.0,-6003.0,Team 1


In [12]:
df_gold_diff = df_gold_diff.set_axis(column_names, axis=1)

In [13]:
df_gold_diff['Winner'] = df_gold_diff.apply(lambda row:
     1 if row['Winner'] == 'Team 1'
     else 0,
     axis=1
)

In [14]:
df_gold_diff["Winner"].values

array([0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1,

In [15]:
x = df_gold_diff.iloc[:, :-1].values
y = df_gold_diff.iloc[:, -1:].values

In [16]:
print(x)
print(y)

[[    0.     0.    34. ...  9169. 11533. 10730.]
 [    0.     0.   554. ...  3835.  4840.  3968.]
 [    0.     0.  1513. ...  9531.  8937.  8863.]
 ...
 [    0.     0.    12. ...   731. -1992.  -467.]
 [    0.   519.   518. ... -5912. -6809. -6003.]
 [    0.     0.    41. ... -3280. -4869. -7697.]]
[[0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]


In [17]:
tb_callback = tf.keras.callbacks.TensorBoard(log_dir="logs/", histogram_freq=1)

In [18]:
import tensorflow as tf
nn = tf.keras.models.Sequential()

In [19]:
nn.add(tf.keras.layers.Dense(units=21, activation="relu"))

In [20]:
nn.add(tf.keras.layers.Dense(units=4, activation="relu"))

In [21]:
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

In [22]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [23]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)

In [24]:
nn.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [25]:
nn.fit(x_train, y_train, batch_size=64, epochs=100, callbacks=[tb_callback])

Epoch 1/100
9/9 [==============================] - 1s 9ms/step - loss: 0.8847 - accuracy: 0.4330
Epoch 2/100
9/9 [==============================] - 0s 9ms/step - loss: 0.7654 - accuracy: 0.4696
Epoch 3/100
9/9 [==============================] - 0s 9ms/step - loss: 0.6926 - accuracy: 0.5443
Epoch 4/100
9/9 [==============================] - 0s 7ms/step - loss: 0.6510 - accuracy: 0.6122
Epoch 5/100
9/9 [==============================] - 0s 8ms/step - loss: 0.6312 - accuracy: 0.6678
Epoch 6/100
9/9 [==============================] - 0s 7ms/step - loss: 0.6160 - accuracy: 0.6939
Epoch 7/100
9/9 [==============================] - 0s 8ms/step - loss: 0.5995 - accuracy: 0.7165
Epoch 8/100
9/9 [==============================] - 0s 7ms/step - loss: 0.5779 - accuracy: 0.7287
Epoch 9/100
9/9 [==============================] - 0s 7ms/step - loss: 0.5516 - accuracy: 0.7357
Epoch 10/100
9/9 [==============================] - 0s 9ms/step - loss: 0.5276 - accuracy: 0.7443
Epoch 11/100
9/9 [===========

In [26]:
y_pred = nn.predict(x_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[1 1]
 [1 0]
 [1 1]
 [1 1]
 [0 0]
 [1 1]
 [0 0]
 [1 0]
 [1 1]
 [0 0]
 [0 0]
 [0 0]
 [0 1]
 [1 1]
 [1 1]
 [0 0]
 [0 1]
 [1 1]
 [0 0]
 [1 0]
 [1 1]
 [1 1]
 [1 1]
 [0 1]
 [0 0]
 [0 0]
 [0 1]
 [1 1]
 [0 0]
 [1 1]
 [0 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [1 1]
 [1 0]
 [1 1]
 [1 1]
 [1 1]
 [0 0]
 [1 1]
 [1 0]
 [0 0]
 [0 0]
 [1 1]
 [1 1]
 [1 0]
 [1 1]
 [0 0]
 [0 1]
 [1 1]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 1]
 [1 1]
 [0 1]
 [0 1]
 [0 0]
 [0 0]
 [0 0]
 [0 1]
 [1 1]
 [0 0]
 [0 1]
 [1 0]
 [0 0]
 [0 0]
 [1 1]
 [1 1]
 [1 1]
 [0 0]
 [1 1]
 [1 1]
 [0 0]
 [1 1]
 [1 1]
 [0 0]
 [1 1]
 [0 0]
 [1 1]
 [1 1]
 [1 1]
 [1 0]
 [0 0]
 [1 1]
 [0 1]
 [1 1]
 [0 1]
 [0 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [0 0]
 [0 0]
 [1 1]
 [1 1]
 [0 0]
 [1 1]
 [0 1]
 [1 0]
 [1 1]
 [1 1]
 [1 1]
 [0 1]
 [1 0]
 [0 0]
 [0 1]
 [1 1]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [0 1]
 [0 0]
 [0 0]
 [1 1]
 [1 1]
 [0 0]
 [0 1]
 [1 0]
 [1 1]
 [0 0]
 [0 0]
 [1 0]
 [0 1]
 [1 1]
 [0 0]
 [0 1]
 [0 0]

In [27]:
results = nn.evaluate(x_test, y_test, batch_size=128)
print("test loss, test acc:", results)

2/2 [==============================] - 0s 997us/step - loss: 0.5368 - accuracy: 0.7778
test loss, test acc: [0.5368421077728271, 0.7777777910232544]


In [28]:
from tensorboard import notebook
notebook.list()

Known TensorBoard instances:
  - port 6006: logdir logs/ (started 1 day, 6:03:12 ago; pid 7332)


In [30]:
nn.save("model_with_normalize_2")

INFO:tensorflow:Assets written to: model_with_normalize_2\assets
